In [227]:
import networkx as nx
from matplotlib import pyplot as plt
import random
import math
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import pandas as pd
from tqdm import tqdm
import threading
import queue

In [228]:
#mode = 'MF', 'R', 'None'
node_count = 50000
infection_rate = 0.05
recovery_rate = 0.01
contact_radius = 1
time_steps = 80
temporal_graph_dict = {}
observe_per_day = 100
t_mf = 5
delay = 10
mode = 'R'
color_map = {'S': 'green', 'I': 'red', 'R': 'blue'}

In [229]:
SIR_prob_node = {'S':np.ones(node_count), 'I':np.zeros(node_count), 'R':np.zeros(node_count)}
SIR_prob_node['I'][1] = 1
SIR_prob_node['S'][1] = 0

In [230]:
def distance(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def exp_dist(x, y, density, pos_dic):
    a = pos_dic[x]
    b = pos_dic[y]
    return math.exp(-distance(a, b)/density)

In [231]:
def MeanFieldInferenceVector(SIR_prob_nodes_t, State_matrix_t, Adj_matrix_t, infection_rate, recovery_rate):
    #convert to column vector
    P_S_t = np.transpose([SIR_prob_nodes_t['S']])
    P_I_t = np.transpose([SIR_prob_nodes_t['I']])
    P_R_t = np.transpose([SIR_prob_nodes_t['R']])
    #sum of product of neighbors' state and infection rate
    factor = np.dot(State_matrix_t[1], np.dot(Adj_matrix_t,P_I_t))*infection_rate
    P_S_t1 = np.multiply(P_S_t, 1-factor)
    P_R_t1 = P_R_t+ recovery_rate*P_I_t
    P_I_t1 = np.ones((node_count,1)) - P_S_t1 - P_R_t1
    return P_S_t1, P_I_t1, P_R_t1

In [232]:
def initialize_graph(node_count, contact_radius = 1, queue = None, time = 0, edge_dist='geometric'):
    square_size = math.sqrt(node_count) #size of square
    Adj_matrix = np.zeros((node_count, node_count))
    State_matrix = np.zeros((3, node_count))
    #put all nodes in S state
    State_matrix[0] = 1
    position = {}
    for i in range(node_count):
        x,y = random.uniform(0, square_size), random.uniform(0, square_size)
        position[i] = (x,y)
    #edges 
    #check each unique pair of nodes
    if edge_dist == 'exp':
        pairs = np.array(np.triu_indices(3, k=1)).T
        for pair in pairs:
            if random.random() < exp_dist(pair[0], pair[1], contact_radius, position):
                Adj_matrix[pair[0]][pair[1]] = 1
                Adj_matrix[pair[1]][pair[0]] = 1
    elif edge_dist == 'geometric':
        pairs = np.array(np.triu_indices(node_count, k=1)).T
        for pair in pairs:
            if random.random() < math.exp(-distance(position[pair[0]], position[pair[1]])/contact_radius):
                Adj_matrix[pair[0]][pair[1]] = 1
                Adj_matrix[pair[1]][pair[0]] = 1
    return Adj_matrix, State_matrix, position 

In [235]:
Adj_mat, State_matrix, position = initialize_graph(node_count, contact_radius = 1, queue = None, time = 0, edge_dist='exp')